In [1]:
import re
import pandas as pd

# Use the inside_block constraint the searching region (Correct)

In [2]:
out_file = str(input('path of your file'))
Info_after_optimized = []
pattern1 = r'Optimization is complete'
pattern2 = r'Final energy is'
pattern3 = r'Properties computed using the full oxidized Q density matrix'
pattern4 = r'Dipole Moment: \[D\]'
lines_after_p4 = 1
pattern5 = r'Mulliken Charges:'
lines_after_p5=int(input('the numbers of the atoms in your molecules'))+1
pattern6 = r'Total alpha'
pattern7 = r' Harmonic Vibrational Analysis'
pattern8 = r'Freq'

pattern_list = [pattern2, pattern4, pattern5, pattern8]
found_p1, found_p3, found_p6, found_p7 = (False, False, False, False)
# Seperate to 3 parts to read (1st: Final Energy, 2nd: Dipole moment and Mulliken Charge, 3rd: Freq)
inside_block = False
# For reading the middle part (2nd part).

with open(out_file, 'r') as file:
    for line in file:
        if not found_p1:
            match_p1 = re.search(pattern1, line)
            if match_p1:
                found_p1 = True
        # read the file start from the pattern1
        else:
            for pattern in pattern_list:
                match = re.search(pattern, line)
                if match:
                    Info_after_optimized.append(line.strip())
                # find the pattern in pattern_list and search the pattern1 first (stop searching from the later constrant)
                if not found_p3:
                    match_p3 = re.search(pattern3, line)
                    if match_p3:
                        found_p3 = True
                        inside_block = True # Suppose the inside block exist (pattern3 is the beginning of inside block)
                # read the file after the pattern3
                else:
                    if inside_block: # Check if the inside_block exist
                        if re.search(pattern6, line):
                            inside_block = False # This is a bit unintuitive but it is a crucial part for the correctness of the programming
#                           Exit the block when it find out the pattern6, such that the program can search the remaining lines after the block and make sure the following code (search) is correct (not mistake it still in the block) 
                            found_p6 = True  # Set the flag to True after finding pattern6
                            break  # Exit the loop when pattern6 is found
                        for pattern in pattern_list:
                            match = re.search(pattern, line)
                            if match:
                                Info_after_optimized.append(line)
                                if pattern == pattern4:
                                    for _ in range(lines_after_p4):
                                        line = next(file).strip()
                                        Info_after_optimized.append(line)
                                    break
                                if pattern == pattern5:
                                    row_data = []
                                    for _ in range(lines_after_p5):
                                        line = next(file).strip()
                                        row_data.append(line)
                                    Info_after_optimized.append(row_data[1:])
                                    break
                    if found_p6:
                        break 
                        # *the essence of the inside_block*: break the for loop under the condition of inside the block, and cut off the searching process of this for loop when "inside_block=False"(exit the p3 and p6). 

        if not found_p7:
            match_p7= re.search(pattern7,line)
            if match_p7:
                found_p7=True
        else:
            for pattern in pattern_list:
                match = re.search(pattern, line)
                if match:
                    if pattern==pattern8:
                        Info_after_optimized.append(line.strip())
                                

if not found_p1:
    print('Geometry Optimization failed')
elif not found_p3:
    print("SCF density analysis failed")
elif not found_p7:
    print('Frequency calculation failed')
else:
    print('Optimization Complete')

df = pd.DataFrame(Info_after_optimized, columns=['Information after Geometry Optimization'])
pd.set_option('display.max_colwidth', None)

df


path of your file psi4 out.file/glucose.out
the numbers of the atoms in your molecules 24


Optimization Complete


,Information after Geometry Optimization
0,Final energy is -679.5483089245736
1,Dipole Moment: [D]
2,Dipole Moment: [D]\n
3,X: 2.9853 Y: 1.1573 Z: 5.4355 Total: 6.3084
4,Mulliken Charges: (a.u.)\n
5,"[1 C 2.77358 2.77358 0.00000 0.45283, 2 C 2.98425 2.98425 0.00000 0.03150, 3 C 2.98774 2.98774 0.00000 0.02451, 4 O 4.34259 4.34259 0.00000 -0.68518, 5 C 2.98382 2.98382 0.00000 0.03236, 6 H 0.37186 0.37186 0.00000 0.25629, 7 C 2.99492 2.99492 0.00000 0.01016, 8 O 4.34627 4.34627 0.00000 -0.69255, 9 H 0.38909 0.38909 0.00000 0.22183, 10 O 4.33707 4.33707 0.00000 -0.67414, 11 H 0.37520 0.37520 0.00000 0.24960, 12 C 3.04148 3.04148 0.00000 -0.08295, 13 H 0.38063 0.38063 0.00000 0.23874, 14 O 4.35093 4.35093 0.00000 -0.70185, 15 H 0.29765 0.29765 0.00000 0.40470, 16 O 4.36084 4.36084 0.00000 -0.72167, 17 H 0.36846 0.36846 0.00000 0.26308, 18 H 0.28715 0.28715 0.00000 0.42571, 19 H 0.30188 0.30188 0.00000 0.39625, 20 H 0.29863 0.29863 0.00000 0.40274, 21 H 0.38310 0.38310 0.00000 0.23380, 22 O 4.34376 4.34376 0.00000 -0.68752, 23 H 0.39717 0.39717 0.00000 0.20565, 24 H 0.30194 0.30194 0.00000 0.39612]"
6,Freq [cm^-1] 97.8581 104.6158 160.5265
7,Freq [cm^-1] 203.9347 210.3931 217.6809
8,Freq [cm^-1] 249.7459 291.6005 313.7147
9,Freq [cm^-1] 331.2828 352.1306 361.6453


# Not use the inside_block (more intuitive, less lines of code), but should deeply understand the operation of loop.

In [3]:
out_file = str(input('path of your file'))
Info_after_optimized = []
pattern1 = r'Optimization is complete'
pattern2 = r'Final energy is'
pattern3 = r'Properties computed using the full oxidized Q density matrix'
pattern4 = r'Dipole Moment: \[D\]'
lines_after_p4 = 1
pattern5 = r'Mulliken Charges:'
lines_after_p5=int(input('the numbers of the atoms in your molecules'))+1
pattern6 = r'Total alpha'
pattern7 = r' Harmonic Vibrational Analysis'
pattern8 = r'Freq'

pattern_list = [pattern2, pattern4, pattern5, pattern8]
found_p1, found_p3, found_p6, found_p7 = (False, False, False, False)

with open(out_file, 'r') as file:
    for line in file:
        if not found_p1:
            match_p1 = re.search(pattern1, line)
            if match_p1:
                found_p1 = True
        else:
            for pattern in pattern_list:
                match = re.search(pattern, line)
                if match:
                    Info_after_optimized.append(line.strip())
                if not found_p3:
                    match_p3 = re.search(pattern3, line)
                    if match_p3:
                        found_p3 = True
                else:
                    for pattern in pattern_list:
                        if re.search(pattern6, line):
                            found_p6 = True

                        match = re.search(pattern, line)
                        if match:
                            Info_after_optimized.append(line)
                            if pattern == pattern4:
                                for _ in range(lines_after_p4):
                                    line = next(file).strip()
                                    Info_after_optimized.append(line)
                                break
                            if pattern == pattern5:
                                row_data = []
                                for _ in range(lines_after_p5):
                                    line = next(file).strip()
                                    row_data.append(line)
                                Info_after_optimized.append(row_data[1:])
                                break
                        if found_p6:
                            break
                break # break the for loop of the next line of "else:" statement
       
            if not found_p7:
                match_p7= re.search(pattern7,line)
                if match_p7:
                    found_p7=True
            else:
                for pattern in pattern_list:
                    match = re.search(pattern, line)
                    if match:
                        if pattern==pattern8:
                            Info_after_optimized.append(line.strip())
                                

if not found_p1:
    print('Geometry Optimization failed')
elif not found_p3:
    print("SCF density analysis failed")
elif not found_p7:
    print('Frequency calculation failed')
else:
    print('Optimization Complete')

df = pd.DataFrame(Info_after_optimized, columns=['Information after Geometry Optimization'])
pd.set_option('display.max_colwidth', None)

df.head(6)


path of your file psi4 out.file/glucose.out
the numbers of the atoms in your molecules 24


Optimization Complete


,Information after Geometry Optimization
0,Final energy is -679.5483089245736
1,Dipole Moment: [D]\n
2,X: 2.9853 Y: 1.1573 Z: 5.4355 Total: 6.3084
3,Mulliken Charges: (a.u.)\n
4,"[1 C 2.77358 2.77358 0.00000 0.45283, 2 C 2.98425 2.98425 0.00000 0.03150, 3 C 2.98774 2.98774 0.00000 0.02451, 4 O 4.34259 4.34259 0.00000 -0.68518, 5 C 2.98382 2.98382 0.00000 0.03236, 6 H 0.37186 0.37186 0.00000 0.25629, 7 C 2.99492 2.99492 0.00000 0.01016, 8 O 4.34627 4.34627 0.00000 -0.69255, 9 H 0.38909 0.38909 0.00000 0.22183, 10 O 4.33707 4.33707 0.00000 -0.67414, 11 H 0.37520 0.37520 0.00000 0.24960, 12 C 3.04148 3.04148 0.00000 -0.08295, 13 H 0.38063 0.38063 0.00000 0.23874, 14 O 4.35093 4.35093 0.00000 -0.70185, 15 H 0.29765 0.29765 0.00000 0.40470, 16 O 4.36084 4.36084 0.00000 -0.72167, 17 H 0.36846 0.36846 0.00000 0.26308, 18 H 0.28715 0.28715 0.00000 0.42571, 19 H 0.30188 0.30188 0.00000 0.39625, 20 H 0.29863 0.29863 0.00000 0.40274, 21 H 0.38310 0.38310 0.00000 0.23380, 22 O 4.34376 4.34376 0.00000 -0.68752, 23 H 0.39717 0.39717 0.00000 0.20565, 24 H 0.30194 0.30194 0.00000 0.39612]"
5,Freq [cm^-1] 97.8581 104.6158 160.5265
